# Generate Simulation Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PyMca5.PyMca import ConfigDict
from PyMca5.PyMca import ClassMcaTheory
from PyMca5.PyMca import ConcentrationsTool
import pandas as pd
import h5py
import time

In [ ]:
channels = list(range(4096))
spectra = np.loadtxt("Book1.csv", delimiter=',') # load one example spectra with channel for PyMca initialization
plt.plot(channels, spectra)
plt.ylabel('Intensity')
plt.xlabel('Energy Level')

In [ ]:
Pigment_list = ['CaCO3',
                'Cadmium yellow',
                'Chromate yellow',
                'Chromate green',
                'Cobalt blue',
                'Emerald green',
                'Iron oxide',  
                'Lead White',
                'Prussian blue',
                'Red lead',
                'SnO2',
                'Ultramarine blue',
                'Vermilion',
                'ZnO']

Top_Pigment_list = ['Cadmium yellow',
                    'Cobalt blue',
                    'Emerald green',
                    'Iron oxide', 
                    'Lead White', 
                    'Prussian blue', 
                    'SnO2', 
                    'Ultramarine blue',
                    'Vermilion']

Bottom_Pigment_list = ['Chromate green',
                       'Chromate yellow',
                       'Cobalt blue',
                       'Emerald green',
                       'Lead White', 
                       'Red lead',
                       'Ultramarine blue',
                       'Vermilion',
                       'ZnO']

In [ ]:
config_multilayer = ConfigDict.ConfigDict()
config_multilayer.read("GauguinPainting1.cfg")# cfg file path

In [ ]:
config_multilayer['multilayer']

Groud layer: CaCO3 450 um 2g/cm3

In [ ]:
config_multilayer['multilayer']['Layer2'] = [1, 'CaCO3', 2, 0.45] # add the fixed ground layer

In [ ]:
spectra_sheet = pd.DataFrame(columns = list(range(4096)))

classifiers = []
for i in Pigment_list:
    classifiers.append(i + '_top')
    classifiers.append(i + '_bottom')
groudtruth_sheet = pd.DataFrame(columns = classifiers)

## One pigment in one layer
Thickness: 50-200 um for the first layer, 100-150 um for the second layer

In [ ]:
%%time

start = time.time()

classifier_list = np.zeros((1, 28))
mcaFit_multilayer = ClassMcaTheory.McaTheory()
mcaFit_multilayer.setData(channels, spectra)


for i in range(1, 14): # for top-pigment
    top_pigment = Pigment_list[i]
    classifier_list[0, 2*i] = 1
    top_density = 4 # density for all layer is same
    
    for j in range(1, 14): # for bottom-pigment
        bottom_pigment = Pigment_list[j]        
        if (top_pigment in Top_Pigment_list) and (bottom_pigment in Bottom_Pigment_list):
            continue
        bottom_density = 4 # density for all layer is same
        classifier_list[0, 2*j + 1] = 1
            
        for top_thickness in np.arange(0.005, 0.021, 0.001): #top layer thickness: 50 - 200 um
            config_multilayer['multilayer']['Layer0'] = [1, top_pigment, top_density, top_thickness]    

            for bottom_thickness in np.arange(0.01, 0.016, 0.001):  #bottom layer thickness: 100 - 150 um
                gt = pd.DataFrame(classifier_list, columns = classifiers)
                groudtruth_sheet = groudtruth_sheet.append(gt, ignore_index=True)         

                config_multilayer['multilayer']['Layer1'] = [1, bottom_pigment, bottom_density, bottom_thickness]

                mcaFit_multilayer.configure(config_multilayer)
                mcaFit_multilayer.estimate()
                fitresult_multilayer = mcaFit_multilayer.startFit(digest=1)
                fitResult_multilayer = {}
                fitResult_multilayer['result'] = fitresult_multilayer[1]

                tool = ConcentrationsTool.ConcentrationsTool()
                tool.configure()
                ddict = {}
                ddict.update(config_multilayer['concentrations'])
                tool.configure(ddict)
                ddict, info = tool.processFitResult(fitresult=fitResult_multilayer,
                                                        elementsfrommatrix=True,
                                                        addinfo=True)
                #self._concentrationsInfo = info
                groupsList = fitResult_multilayer['result']['groups']
                areas = []
                for group in groupsList:
                    item = group.split()
                    element = item[0]
                    if len(element) >2:
                        areas.append(0.0)
                    else:
                        area = ddict['area'][group]
                        areas.append(area)

                nglobal    = len(fitResult_multilayer['result']['parameters']) - len(groupsList)
                parameters = []
                for k in range(len(fitResult_multilayer['result']['parameters'])):
                    if k < nglobal:
                        parameters.append(fitResult_multilayer['result']['fittedpar'][k])
                    else:
                        parameters.append(areas[k-nglobal])

                xmatrix = fitResult_multilayer['result']['xdata']
                ymatrix = mcaFit_multilayer.mcatheory(parameters,xmatrix)
                ymatrix.shape =  [len(ymatrix),1]

                s = pd.DataFrame(np.transpose(ymatrix), columns = list(range(4096)))
                spectra_sheet = spectra_sheet.append(s, ignore_index=True)
                print(top_thickness*10000, 'um', top_pigment, 'on', bottom_thickness*10000, 'um', bottom_pigment, 'done!')
        classifier_list[0, 2*j + 1] = 0
        end = time.time()
        print(top_pigment, 'on', bottom_pigment, 'done!')
        print('It has been', end - start, 's!')
        spectra_sheet.to_csv(r'spectra_sheet.csv')
        groudtruth_sheet.to_csv(r'groudtruth_sheet.csv')
    classifier_list[0, 2*i] = 0

In [ ]:
spectra_sheet.to_csv(r'spectra_sheet.csv')
groudtruth_sheet.to_csv(r'groudtruth_sheet.csv')

In [ ]:
spectra_sheet

In [ ]:
groudtruth_sheet